<a href="https://colab.research.google.com/github/sugi-san/Paint/blob/main/Paint_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #**1.セットアップ**

# git clone
!git clone https://github.com/sugi-san/Paint.git
%cd Paint

# import function.py
from function import *

# stop warnings
import warnings
warnings.simplefilter('ignore')

# make folder
reset_folder('download')

In [ ]:
#@title #**2.画像の表示**
display_pic('input')

In [ ]:
#@title #**3.油絵の作成**

from inference import run_inference
import os 
import shutil

# ファイルの指定
picture = 'nakata.jpg' #@param {type:"string"}

# フォルダーリセット
reset_folder('pic')
reset_folder('output')

# 解像度調整
fine = False #@param {type:"boolean"}
if fine == True:
  resolution = 1920
  reso = 'fine_'
else:
  resolution = 1280
  reso = ''

shutil.copy('input/'+picture, 'pic/'+picture)
size_opt('pic/'+picture, resolution)


# プロセス作成
run_inference(input_path='pic/'+picture,
              model_path='inference/model.pth',
              output_dir='output/', 
              need_animation=True,  
              resize_h=None,        
              resize_w=None,
              serial=True)          

# 画像表示
from IPython.display import Image, display, clear_output
clear_output()
display(Image('output/'+picture))

# 画像ダウンロード
from google.colab import files
shutil.copy('output/'+picture, 'download/oil_'+reso+picture)
files.download('download/oil_'+reso+picture)

In [ ]:
#@title #**4.動画の作成**

import glob
from PIL import Image
import os 
import shutil
from IPython.display import clear_output

# フレームフォルダ・リセット
reset_folder('output/frame')

# フレームNo.を0から連番に変換
name = os.path.splitext(picture)[0]
files = sorted(glob.glob('output/'+name+'/*.jpg'))
for i, file in enumerate(files): 
    shutil.copy(file, 'output/frame/'+str(i).zfill(6)+'.jpg')

    # 最終フレームを60枚コピー
    if i == len(files)-1:
      for j in range(len(files), len(files)+60):
         shutil.copy(files[-1], 'output/frame/'+str(j).zfill(6)+'.jpg')

print('making video...')
! ffmpeg -y -r 30 -i output/frame/%06d.jpg -pix_fmt yuv420p -loglevel error output/process.mp4
clear_output()

# mp4再生
display_mp4('./output/process.mp4')

# 動画ダウンロード
from google.colab import files
name = os.path.splitext(picture)[0]
shutil.copy('output/process.mp4', 'download/'+reso+name+'.mp4')
files.download('download/'+reso+name+'.mp4')

In [ ]:
#@title #**5.写真のアップロード**

#@markdown -ファイル名（jpg）は半角英数字にしてください

# ルートへ画像をアップロード
from google.colab import files
uploaded = files.upload()           
uploaded = list(uploaded.keys())    

# ルートから指定フォルダーへ移動
for file in uploaded:
    size_opt(file)
    shutil.move(file, 'input')